In [3]:
import pandas as pd
import numpy as np
import os
import cv2
#import tensorflow as tf
import glob
import h5py
import idx2numpy as idx
import warnings
import re
from PIL import Image
from tqdm import tqdm
# from progress.bar import IncrementalBar

# from data_generator.object_detection_2d_data_generator import DataGenerator
# import xml.etree.ElementTree as ET

In [27]:
warnings.filterwarnings('ignore')

#входное разрешение модели
IMAGE_HEIGHT = 28 # Height of the input images
IMAGE_WIDTH = 28 #Width of the input images
WORK_DIR = 'D:\work\\test_comp_vision\datasets\!_lines_w25_parsed_full' #путь к папке с файлами
DATASET_H5 = 'D:\work\\test_comp_vision\datasets\!_lines_w25_dataset.h5'
DATASET_IDX_IMG = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_images_150k.idx'
DATASET_IDX_CLS = r'D:\work\test_comp_vision\datasets\!_lines_w25_dataset_classes_150k.idx'
DATASET_SIZE = 100000      # какую часть изображений используем для создания датасета
LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
EXPORT_NAME_CSV = '!img_classes.csv'

In [10]:
"""
Структура папок:
base_folder
    image_0
        image_0_0.jpg
        image_0_0.txt
        image_0_1.jpg
        image_0_1.txt
        ...
    image_1
        image_1_0.jpg
        image_1_0.txt
        image_1_1.jpg
        image_1_1.txt
        ...
        
"""
# TODO заменить работу с индексами над путями к файлам на нормальную обработку разрешения файлов
def get_dataframe(path):
    # pattern = r'/^(.+)(\.[^ .]+)?$/'      # для удаления разрешения из пути к файлу
    
    symbol_classes = []
    for folder in os.listdir(path):
        # TODO добавить ограничение на количество итераций здесь, чтобы не создавать DataSet больше нужного
        for img_file in glob.glob(f'{path}/{folder}/*.jpg'):
            txt_file =  f"{img_file[:-3]}txt"    # меняем разрешение для открытия текста [:-3]
            # print(f'IMG_file: {img_file}\nTXT_file: {txt_file}')
            with open(txt_file, 'r') as f:
                symbol_class = f.read()
                
            value = (img_file.replace('\\', '/'), symbol_class)  # меняем системный флеш Винды на тот, что понимает Python
            symbol_classes.append(value)
    column_names = ['file_name', 'class']
    return pd.DataFrame(symbol_classes, columns=column_names)

In [28]:
#Создаем DataFrame в котором хранится путь к фалу/изображению и его класс
base_path = WORK_DIR #путь к основной папке
df_text = pd.DataFrame()

if DATASET_SIZE > 0:
    df_text = df_text.append(get_dataframe(base_path)[:DATASET_SIZE])
else:
    df_text = df_text.append(get_dataframe(base_path))
df_text

,file_name,class
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,в
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,е
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,е
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,р
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,а
...,...,...
99995,D:/work/test_comp_vision/datasets/!_lines_w25_...,С
99996,D:/work/test_comp_vision/datasets/!_lines_w25_...,Т
99997,D:/work/test_comp_vision/datasets/!_lines_w25_...,Р
99998,D:/work/test_comp_vision/datasets/!_lines_w25_...,О


In [29]:
# Преобразуем классы к числовым значениям и меняем формат в ссылках на unix-like (иначе PIL не может открыть файл)
labels_to_int = {}
for i, label in enumerate(LABELS):
    labels_to_int[label] = i
print(labels_to_int)

df_images = pd.DataFrame({'image_path':df_text['file_name'],
                   'class_id':[labels_to_int[label] for label in df_text['class']]})
# df_images.to_csv(EXPORT_NAME_CSV, index=False)
df_images

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'А': 10, 'Б': 11, 'В': 12, 'Г': 13, 'Д': 14, 'Е': 15, 'Ё': 16, 'Ж': 17, 'З': 18, 'И': 19, 'Й': 20, 'К': 21, 'Л': 22, 'М': 23, 'Н': 24, 'О': 25, 'П': 26, 'Р': 27, 'С': 28, 'Т': 29, 'У': 30, 'Ф': 31, 'Х': 32, 'Ц': 33, 'Ч': 34, 'Ш': 35, 'Щ': 36, 'Ъ': 37, 'Ы': 38, 'Ь': 39, 'Э': 40, 'Ю': 41, 'Я': 42, 'а': 43, 'б': 44, 'в': 45, 'г': 46, 'д': 47, 'е': 48, 'ё': 49, 'ж': 50, 'з': 51, 'и': 52, 'й': 53, 'к': 54, 'л': 55, 'м': 56, 'н': 57, 'о': 58, 'п': 59, 'р': 60, 'с': 61, 'т': 62, 'у': 63, 'ф': 64, 'х': 65, 'ц': 66, 'ч': 67, 'ш': 68, 'щ': 69, 'ъ': 70, 'ы': 71, 'ь': 72, 'э': 73, 'ю': 74, 'я': 75}


,image_path,class_id
0,D:/work/test_comp_vision/datasets/!_lines_w25_...,45
1,D:/work/test_comp_vision/datasets/!_lines_w25_...,48
2,D:/work/test_comp_vision/datasets/!_lines_w25_...,48
3,D:/work/test_comp_vision/datasets/!_lines_w25_...,60
4,D:/work/test_comp_vision/datasets/!_lines_w25_...,43
...,...,...
99995,D:/work/test_comp_vision/datasets/!_lines_w25_...,28
99996,D:/work/test_comp_vision/datasets/!_lines_w25_...,29
99997,D:/work/test_comp_vision/datasets/!_lines_w25_...,27
99998,D:/work/test_comp_vision/datasets/!_lines_w25_...,25


## .H5 файл генератор (не используем, т.к. проще было пристроить .IDX)

In [1]:
#Создаем файл датасета для обучения детектора
#train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
#images_dir = ''

#train_labels_filename = EXPORT_NAME_CSV

#train_dataset.parse_csv(images_dir=images_dir,
#                        labels_filename=train_labels_filename,
#                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
#                        #input_format=['image_name', 'class_id'],
#                        include_classes='all')

#train_dataset.create_hdf5_dataset(file_path=DATASET_H5,
#                                  resize=[IMAGE_HEIGHT, IMAGE_WIDTH],
#                                  variable_image_size=True,
#                                  verbose=True)

In [30]:
print(df_images.shape)
for i, row in df_images[:5].iterrows():  
    print(row['class_id'])

(100000, 2)
45
48
48
60
43


## .IDX файл генератор

In [12]:
with Image.open(r"D:\work\test_comp_vision\test_for_MindSet\pass_temp\0\0\0-0.jpg") as image:
        print(image.size)
        parsed_image = np.asarray(image)
        print(parsed_image.shape)
        parsed_image2 = np.asarray([parsed_image])
        print(parsed_image2.shape)
        

(28, 28)
(28, 28)
(1, 28, 28)


In [31]:
# преобразуем все изображения в массив np.array()

parsed_images = []              # создаем два файла: в одном хранятся изображения в формате np.array
parsed_classes = []             # во втором хранятся классы
print(f"Общее количество: {df_images.shape[0]}")

for i, row in tqdm(df_images.iterrows()):
    
    with Image.open(row['image_path']) as image:
        parsed_images.append(np.asarray(image))  #открываем картинку по ссылке, преобразуем в массив
    #print(parsed_images[row[0]].shape)
    
    parsed_classes.append(row['class_id'])
    #print(parsed_classes[row[0]])

np_images = np.asarray(parsed_images) 
np_classes = np.asarray(parsed_classes)

print(f"Датасет изображений к сохранению: {np_images.shape}")
print(f"Датасет классов к сохранению: {np_classes.shape}")

idx.convert_to_file(DATASET_IDX_IMG, np_images)
idx.convert_to_file(DATASET_IDX_CLS, np_classes)




Общее количество: 100000


100000it [04:46, 349.59it/s]


Датасет изображений к сохранению: (100000, 28, 28)
Датасет классов к сохранению: (100000,)


In [32]:
images = idx.convert_from_file(DATASET_IDX_IMG)
classes = idx.convert_from_file(DATASET_IDX_CLS)

print(f"Датасет изображений из файла: {images.shape}")
print(f"Датасет классов из файла: {classes.shape}")

Датасет изображений из файла: (100000, 28, 28)
Датасет классов из файла: (100000,)


In [33]:
#images = idx.convert_from_file(DATASET_IDX_IMG)
classes = idx.convert_from_file(r"D:\work\test_comp_vision\datasets\emnist_gzip\emnist-byclass-train-labels-idx1-ubyte")

#print(f"Датасет изображений из файла: {images.shape}")
print(f"Датасет классов из файла: {classes.shape}")

Датасет классов из файла: (697932,)
